# Importing libraries

In [1]:
import pandas as pd
import pymysql
from sqlalchemy import create_engine

# Changing the way pandas displays floats

# Importing dataset

In [2]:
suicides = pd.read_csv('/Users/andressalomferrer/Desktop/ironhack/Projects/Project_3/Project-Week-3-Data-Thieves/your-project/suicides_cleaned')

In [3]:
suicides['Population'].sum()

45776791044

In [4]:
suicides.Year.unique()

array([1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002,
       2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 1990, 1991, 2012,
       2013, 2014, 2015, 2011])

# QUESTION 4:

# 1. MALES OR FEMALES COMMIT SUICIDE MORE?

In [5]:
suicides.dtypes

Country              object
Year                  int64
Gender               object
Age                  object
Num_Suicides          int64
Population            int64
Suicides/100kPop    float64
GDP/Year              int64
GDP/Capita            int64
Generation           object
dtype: object

In [6]:
suicides = suicides.astype({"Year": 'object'})

In [7]:
# From 1990 till 2015

In [8]:
tot_s_mf = suicides[['Gender','Num_Suicides','Year']].groupby('Gender').sum().reset_index()

In [9]:
tot_s_mf['Percentage'] = tot_s_mf['Num_Suicides'].apply(lambda row: (row/sum(tot_s_mf['Num_Suicides'])*100))

In [10]:
tot_s_mf = tot_s_mf.round({'Percentage': 2})

## 1. Solution: males  77.3%, females: 22.7%

In [11]:
tot_s_mf

,Gender,Num_Suicides,Percentage
0,female,1380293,22.67
1,male,4707679,77.33


# 2. Males or females commit suicide more in 2015?

In [12]:
# To answer this question we are going to create a dataframe for the year 2015:

suicides_2015 = suicides[suicides['Year'] == 2015]

In [13]:
# Now we are going to filter by the columns we need:

s_gender2015 = suicides_2015[['Gender','Num_Suicides']].groupby('Gender').sum().reset_index()

## As we can see, the number of suicides in 2015 is higher in males than in females:


In [14]:
s_gender2015

,Gender,Num_Suicides
0,female,47248
1,male,156392


In [15]:
s_gender2015['Percentage'] = s_gender2015['Num_Suicides'].apply(lambda row: (row/sum(s_gender2015['Num_Suicides'])*100))

In [16]:
s_gender2015

,Gender,Num_Suicides,Percentage
0,female,47248,23.201729
1,male,156392,76.798271


In [17]:
s_gender2015 = s_gender2015.round({'Percentage': 2})

# 2. Solution: males 76.8%, females 23.2%

In [18]:
s_gender2015

,Gender,Num_Suicides,Percentage
0,female,47248,23.2
1,male,156392,76.8


# 3. TOP 10 COUNTRIES BY GENRE:

## BY NUMBER:

### Top 10 countries with more suicides in year 2015 taking into account the number of suicides.

In [19]:
top10countriesbynum = suicides_2015[['Country','Num_Suicides']].groupby('Country').sum().sort_values("Num_Suicides", ascending=False)[0:10]

In [20]:
top10countriesbynum

,Num_Suicides
Country,
United States,44189
Russian Federation,25432
Japan,23092
Republic of Korea,13510
Brazil,11163
Germany,10088
Ukraine,7574
Mexico,6234
Poland,5420


### MALES : TOP 10 COUNTRIES BY NUM

In [21]:
top10malesbynum = suicides_2015[['Country','Gender','Num_Suicides']].loc[suicides_2015['Gender']=='male'].groupby('Country').sum().sort_values("Num_Suicides", ascending=False)[0:10]

In [22]:
top10malesbynum

,Num_Suicides
Country,
United States,33990
Russian Federation,20861
Japan,16146
Republic of Korea,9556
Brazil,8766
Germany,7403
Ukraine,6148
Mexico,4986
Poland,4697


### FEMALES : TOP 10 COUNTRIES BY NUM

In [23]:
top10femalesbynum = suicides_2015[['Country','Gender','Num_Suicides']].loc[suicides_2015['Gender']=='female'].groupby('Country').sum().sort_values("Num_Suicides", ascending=False)[0:10]

In [24]:
top10femalesbynum

,Num_Suicides
Country,
United States,10199
Japan,6946
Russian Federation,4571
Republic of Korea,3954
Germany,2685
Brazil,2397
Ukraine,1426
Mexico,1248
United Kingdom,1178


In [25]:
# Creating an intersection by index to see if the countries with a higher number of suicides
# are the same countries for both genres.

idgnf = pd.Index(top10femalesbynum.index)
idgnm = pd.Index(top10malesbynum.index)

In [26]:
set_intersection = idgnf.intersection(idgnm)

In [27]:
set_intersection

Index(['United States', 'Japan', 'Russian Federation', 'Republic of Korea',
       'Germany', 'Brazil', 'Ukraine', 'Mexico', 'United Kingdom'],
      dtype='object', name='Country')

### As we can see 9 out of 10 countries match

In [28]:
# Spain appears in top10 of countries where females commit suicides.

idgnf.difference(idgnm)

Index(['Spain'], dtype='object', name='Country')

In [29]:
# Poland appears in top10 of countries where males commit suicides.

idgnm.difference(idgnf)

Index(['Poland'], dtype='object', name='Country')

In [30]:
set_intersection = idgnf.intersection(idgnm)

In [31]:
set_intersection

Index(['United States', 'Japan', 'Russian Federation', 'Republic of Korea',
       'Germany', 'Brazil', 'Ukraine', 'Mexico', 'United Kingdom'],
      dtype='object', name='Country')

## Now we realize that took into account the num of suicides. But if we take into account the size of the population, results can be different. Let's check:


## BY SIZE:

In [32]:
top10bysize = suicides_2015[['Country','Suicides/100kPop']].groupby('Country').sum().sort_values("Suicides/100kPop", ascending=False)[0:10]

In [33]:
top10bysize

,Suicides/100kPop
Country,
Republic of Korea,389.14
Lithuania,385.48
Uruguay,270.02
Slovenia,264.81
Hungary,257.27
Latvia,251.47
Ukraine,244.72
Kazakhstan,238.02
Russian Federation,236.92


In [34]:
# Creating an intersection by index to see which countries appears in both rankings.
idx1 = pd.Index(top10bysize.index)
idx2 = pd.Index(top10countriesbynum.index)

In [35]:
set_intersection

Index(['United States', 'Japan', 'Russian Federation', 'Republic of Korea',
       'Germany', 'Brazil', 'Ukraine', 'Mexico', 'United Kingdom'],
      dtype='object', name='Country')

### MALE: TOP 10 COUNTRIES BY SIZE

In [36]:
top10malesbysize = suicides_2015[['Country','Gender','Suicides/100kPop']].loc[suicides_2015['Gender']=='male'].groupby('Country').sum().sort_values("Suicides/100kPop", ascending=False)[0:10]

In [37]:
top10malesbysize

,Suicides/100kPop
Country,
Lithuania,323.31
Republic of Korea,285.19
Uruguay,223.03
Slovenia,217.67
Latvia,211.07
Ukraine,206.64
Hungary,203.39
Russian Federation,199.26
Kazakhstan,195.35


### FEMALE: TOP 10 COUNTRIES BY SIZE

In [38]:
top10femalesbysize = suicides_2015[['Country','Gender','Suicides/100kPop']].loc[suicides_2015['Gender']=='female'].groupby('Country').sum().sort_values("Suicides/100kPop", ascending=False)[0:10]

In [39]:
top10femalesbysize

,Suicides/100kPop
Country,
Republic of Korea,103.95
Lithuania,62.17
Japan,58.29
Hungary,53.88
Belgium,52.21
Croatia,51.63
Slovenia,47.14
Uruguay,46.99
Iceland,45.69


In [40]:
# Creating an intersection by index to see if the countries with a higher number of suicides 
# by size of the population are the same countries for both genres.

idgsf = pd.Index(top10femalesbysize.index)
idgsm = pd.Index(top10malesbysize.index)

In [41]:
set_intersection = idgsf.intersection(idgsm)

In [42]:
set_intersection

Index(['Republic of Korea', 'Lithuania', 'Hungary', 'Croatia', 'Slovenia',
       'Uruguay'],
      dtype='object', name='Country')

In [43]:
# females but no males:

idgsf.difference(idgsm)

Index(['Belgium', 'Iceland', 'Japan', 'Serbia'], dtype='object', name='Country')

In [44]:
# males but not females:

idgsm.difference(idgsf)

Index(['Kazakhstan', 'Latvia', 'Russian Federation', 'Ukraine'], dtype='object', name='Country')

### In this case, the countries Republic of Korea,  Lithuania, Hungary, Croatia, Slovenia and Uruguay match for both genres.

### But Belgiun, Iceland, Japan and Serbia appear in the top 10 for females while Kazakhstan, Latvia, Russian Federation and Ukraine appear in the top 10 for males.

# QUESTION 5: 

# 1.  Which generation has committed suicide the most/least?


## First, we will see which generation committed more suicides from 1990 till 2015:

## BY NUMBER:

In [45]:
df_1 = suicides[['Generation','Num_Suicides']].groupby('Generation').sum().sort_values('Num_Suicides', ascending = False)

In [46]:
df_1

,Num_Suicides
Generation,
Boomers,2164259
Silent,1573621
Generation X,1435850
Millenials,620194
G.I. Generation,278142
Generation Z,15906


### We see that in number of suicides, 'Boomers Generation' is the generation than committed the highest number of suicides. What will happen if we divide the number of suicides per generation by the total population for each generation? Let's check:

In [47]:
df_2 = suicides[['Generation','Num_Suicides', 'Suicides/100kPop']].groupby('Generation').sum().sort_values('Suicides/100kPop', ascending = False)

In [48]:
df_2 = df_2.round({'Suicides/100kPop': 2})

In [49]:
df_2

,Num_Suicides,Suicides/100kPop
Generation,,
Silent,1573621,110193.81
Boomers,2164259,67437.74
Generation X,1435850,62763.35
G.I. Generation,278142,45720.77
Millenials,620194,30889.74
Generation Z,15906,944.18


# in 2015:

In [50]:
# checking the data 

suicides_2015.head()

,Country,Year,Gender,Age,Num_Suicides,Population,Suicides/100kPop,GDP/Year,GDP/Capita,Generation
480,Antigua and Barbuda,2015,female,55-74 years,1,6403,15.62,1364863037,14853,Boomers
481,Antigua and Barbuda,2015,female,15-24 years,0,8561,0.00,1364863037,14853,Millenials
482,Antigua and Barbuda,2015,female,25-34 years,0,7740,0.00,1364863037,14853,Millenials
483,Antigua and Barbuda,2015,female,35-54 years,0,15323,0.00,1364863037,14853,Generation X
484,Antigua and Barbuda,2015,female,5-14 years,0,8239,0.00,1364863037,14853,Generation Z


In [51]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)

### Let's see which generation committes more suicides in number of suicides:

In [52]:
suicides_15 = suicides_2015[['Generation','Num_Suicides', 'Population']]

In [53]:
suicides_15.groupby(['Generation']).sum().sort_values('Num_Suicides', ascending = False)

,Num_Suicides,Population
Generation,,
Generation X,71298,515972039
Millenials,53959,540924338
Boomers,53588,354566127
Silent,23114,116559095
Generation Z,1681,246636333


### We see that Silent and Z Generations commit the less suicides. This make sense because Silent Generation is the oldest one and generation Z is the youngest one so, probably, the population is lower than other range of ages. Let's calculate Suicide per generation divided by the total population:

In [54]:
suicides15 =suicides_15.groupby(['Generation']).sum()

In [55]:
suicides15['Sui/Pop'] = (suicides15['Num_Suicides']/suicides15['Population'])*1000000

In [56]:
suicides15.sort_values('Sui/Pop', ascending = False)

,Num_Suicides,Population,Sui/Pop
Generation,,,
Silent,23114,116559095,198.30284
Boomers,53588,354566127,151.13683
Generation X,71298,515972039,138.18191
Millenials,53959,540924338,99.75332
Generation Z,1681,246636333,6.81570


### In terms of population, we see that Silent Generation committes more suicides followed by the Boomer generation.

# QUESTION 6:

# Countries with the highest 5-14 years range suicide rate?

## From 1990 to 2015:

In [136]:
bbs = suicides[suicides['Age'] == '5-14 years']

In [137]:
bbs = bbs[['Country','Suicides/100kPop', 'Population','Num_Suicides']].groupby('Country').sum().sort_values('Suicides/100kPop', ascending = False)

In [138]:
bbs.head()

,Suicides/100kPop,Population,Num_Suicides
Country,,,
Kazakhstan,135.51000,74698751,1911
Suriname,119.01000,2273930,59
Guyana,97.01000,4042921,85
Russian Federation,92.78000,466727825,8529
Kyrgyzstan,85.43000,28451472,469


In [140]:
bbs['Rate'] = bbs['Num_Suicides']/bbs['Suicides/100kPop']

In [143]:
bbs.sort_values('Rate', ascending = False)[0:10].head()

,Suicides/100kPop,Population,Num_Suicides,Rate
Country,,,,
United States,38.17000,1033753232,7656,200.57637
Brazil,15.63000,901547203,2701,172.80870
Mexico,31.62000,587970863,3638,115.05376
Philippines,4.87000,293383160,493,101.23203
Russian Federation,92.78000,466727825,8529,91.92714


# POR QUE ME DA DIFERENTE???

# Countries with the highest 5-14 years range suicide rate in 2015?

In [163]:
bbs15 = suicides.loc[(suicides['Age'] == '5-14 years') & (suicides['Year'] == 2015)]

In [164]:
bbs15 =  bbs15[['Country','Num_Suicides','Population', 'Suicides/100kPop']].groupby('Country').sum().sort_values('Suicides/100kPop', ascending = False)


In [165]:
bbs15 = bbs15.rename(columns={"Population": "Pop_5to14"})

In [166]:
bbs15['Rate'] = bbs15['Num_Suicides']/bbs15['Pop_5to14']

In [170]:
bbs15 = bbs15.sort_values('Rate', ascending = False)[0:10]

In [171]:
bbs15

,Num_Suicides,Pop_5to14,Suicides/100kPop,Rate
Country,,,,
Saint Vincent and Grenadines,1,18337,11.02000,0.00005
Belize,3,77150,7.68000,0.00004
Kyrgyzstan,25,1117425,4.44000,0.00002
Ecuador,62,3074646,4.02000,0.00002
Lithuania,5,273621,3.68000,0.00002
Luxembourg,1,62465,3.29000,0.00002
Kazakhstan,43,2816250,3.02000,0.00002
Estonia,2,139363,2.87000,0.00001
Ukraine,52,4141908,2.49000,0.00001


# POR QUE ME DA DIFERENTE???

# QUESTION 6 Suicide/Population rate per country?

### TOP10 COUNTRIES WITH HIGHER RATE OF SUICIDES FROM 1990-2015:

In [ ]:
s_ratecountry = suicides[['Country','Num_Suicides','Population']].groupby('Country').sum()

In [92]:
s_ratecountry['Rate'] = (s_ratecountry['Num_Suicides']/s_ratecountry['Population'])*1000000

In [94]:
s_ratecountry.sort_values('Rate', ascending = False)[0:10]

,Num_Suicides,Population,Rate
Country,,,
Lithuania,27217,65639914,414.64101
Russian Federation,1171821,3555358820,329.59289
Belarus,59892,197372292,303.44685
Hungary,72130,240253886,300.22407
Sri Lanka,44628,154567226,288.72874
Latvia,12770,44852640,284.71011
Kazakhstan,101546,377513869,268.98614
Slovenia,10615,40268619,263.60477
Estonia,7034,27090810,259.64525


### TOP10 COUNTRIES WITH HIGHER RATE OF SUICIDES IN 2015:

In [101]:
s_ratecountry15 = suicides[suicides['Year'] == 2015]

In [102]:
s_ratecountry15 = s_ratecountry15[['Country','Num_Suicides','Population']].groupby('Country').sum()

In [103]:
s_ratecountry15['Rate'] = (s_ratecountry15['Num_Suicides']/s_ratecountry15['Population'])*1000000

In [105]:
s_ratecountry15.sort_values('Rate', ascending = False)[0:10]

,Num_Suicides,Population,Rate
Country,,,
Lithuania,896,2753926,325.35370
Republic of Korea,13510,48671752,277.57374
Slovenia,422,1954632,215.89742
Latvia,387,1875522,206.34255
Hungary,1868,9389519,198.94523
Uruguay,630,3190795,197.44296
Japan,23092,120376310,191.83176
Ukraine,7574,40345446,187.72875
Russian Federation,25432,137018943,185.60937


In [68]:
df1 = suicides_9015.groupby('Generation').sum().sort_values("Num_Suicides", ascending=False)

NameError: name 'suicides_9015' is not defined

In [ ]:
df1['Gen_sui/tot_sui'] = df1['Num_Suicides'].apply(lambda row: (row/sum(df1['Num_Suicides'])*100))

In [ ]:
df1_f = df1[['Num_Suicides','Gen_sui/tot_sui']].round({'Gen_sui/tot_sui': 2})

In [ ]:
pop_gen = suicides_9015[['Generation','Population']].groupby('Generation').sum().sort_values('Population', ascending = False)

In [ ]:
pop_gen[''] = df1['Num_Suicides'].apply(lambda row: (row/sum(df1['Num_Suicides'])*100))

In [ ]:
df2 = df.groupby('Generation').sum().sort_values("Num_Suicides", ascending=False)

In [ ]:
df2 = df2[['Population', 'Suicides/100kPop']]

In [ ]:
df2['Population'].sum()

In [ ]:
suicides.head()

In [ ]:
suicides_lrange = suicides

In [ ]:
suicides.groupby('Year')['Country'].unique().apply(lambda x: len(x))

In [ ]:
suicides.groupby(['Country','Year']).sum().loc[('Spain',2012)]